# most fundamental chatbot tutorial (great for testing individual features)
This page will describe how to build a very basic chatbot that will enable you to test basic features. This is useful for forinstance coreference resolution, making sure the feature actually works.

In [3]:
# required libraries
import spacy
import neuralcoref

# displacy is not required, but very useful for visualizing results.
from spacy import displacy

ModuleNotFoundError: No module named 'neuralcoref'

## potential issues installing spaCy and neuralcoref
neuralcoref and spaCy are libraries that are still in development. Although they can be used in production, we need good version control to make sure they work well together.

- good guide on installing neuralcoref: https://sunil-guglani.medium.com/a-quick-guide-to-installing-neuralcoref-python-library-90d3c341aaa1
- tip: install neuralcoref from source: https://github.com/huggingface/neuralcoref

tip: install both the English and your desired language model for testing
- python -m spacy download en_core_web_md
- python -m spacy download nl_core_news_lg

### things to check:
- make sure spacy == 2.x if you use a v2.x language model
- make sure visual C++ sdk is installed: https://visualstudio.microsoft.com/visual-cpp-build-tools/
- make sure visual C++ 14 or higher is installed:https://learn.microsoft.com/en-US/cpp/windows/latest-supported-vc-redist?view=msvc-170

### most valuable docs:
- neurcoref intro: https://reposhub.com/python/deep-learning/huggingface-neuralcoref.html
- research doc i want: https://www.researchgate.net/figure/Results-from-NeuralCoref-with-our-example_fig1_346306093
- Great intro to coreference: explosion.ai/blog/coref
- example chatbot with neuralcoref: https://www.kaggle.com/code/wangmo/neuralcoref-jeff-demo

In [2]:
# how to install neuralcoref from source
git clone https://github.com/huggingface/neuralcoref.git
cd neuralcoref
pip install -r requirements.txt
pip install neuralcoref --no-binary neuralcoref
python setup.py build_ext --inplace
pip install -e .
python -m spacy download en_core_web_sm

SyntaxError: invalid syntax (1400255882.py, line 2)

In [ ]:
def answerEnglish(question):
    print('Question: ', question)
    global session_context
    start_pos = len(session_context)
    session_context += (question + " ")
    print("context:",session_context)
    if question in QA: 
        return QA[question]
    else:
        doc = nlp(session_context)
        if doc._.has_coref:
            print("core_clusters", doc._.coref_clusters)
            to_replace = []
            for clust in doc._.coref_clusters:
                main_mention = clust.main
                for mention in clust.mentions:
                    beg, end = mention.start_char - start_pos, mention.end_char - start_pos
                    print("end: ", end)
                    if end > 0:                                     
                        if mention.text in ["its","his","her","my","your","our","their"]:
                            to_replace.append((beg,end,main_mention.text+"'s"))
                        else:
                            to_replace.append((beg,end,main_mention.text))
            to_replace = sorted(to_replace)                         
            print("to_replace:", to_replace)
            question2 = my_coref(question,to_replace)
            print("Coreferenced question:",question2)
            if question2 in QA:
                return QA[question2]
        return "I don't know."

def my_coref(orig_text, to_replace):
    left = 0
    processed_text = ""
    for beg,end,mention in to_replace:
        processed_text += orig_text[left:beg] + mention
        left = end
    processed_text += orig_text[left:]
    return processed_text

In [ ]:
# load the answerEnglish and mycoref functions before these four lines
session_context = ""
answerEnglish("Who is Abraham?")
answerEnglish("When was he born?")